In [371]:
import numpy as np
import re 
from operator import itemgetter

In [372]:
def def_matriz(seq_vert, seq_horiz):

    num_col = len(seq_vert) + 1
    num_lin = len(seq_horiz) + 1
    matriz = np.zeros((num_col, num_lin))

    return matriz

In [373]:

def read_arq():
    with open('input.txt', 'r') as arquivo:
        linhas = arquivo.readlines()
    seq_vert = linhas[0].strip()
    seq_horiz = linhas[1].strip()
    GAP = linhas[2].strip()
    mismatch = linhas[3].strip()
    match = linhas[4].strip()
    return seq_vert, seq_horiz, GAP, mismatch, match


In [374]:
def comp_sequence(matriz, seq_vert, seq_horiz, mismatch, match, i, j):
    if seq_vert[i-1] == seq_horiz[j-1]:
        return int(matriz[i-1][j-1]) + int(match)
    else:
        return int(matriz[i-1][j-1]) + int(mismatch)
        

In [375]:
def smith_waterman(matriz, seq_vert, seq_horiz, GAP, mismatch, match):
    GAP = int(GAP)
    mismatch = int(mismatch)
    match = int(match)
    num_lin = len(matriz)
    num_col = len(matriz[0])

    sum = 0

    matriz_back = def_matriz(seq_vert, seq_horiz)
    #set GAP first column
    for i in range(num_lin):
        for j in range(num_col):
            if j == 0 and i != 0:
                matriz_back[i][j] = 2
    
    #set GAP first line
    for i in range(num_lin):
        for j in range(num_col):
            if i == 0 and j != 0:
                matriz_back[i][j] = 2

    #set GAP first column
    for i in range(num_lin):
        for j in range(num_col):
            if j == 0 and i != 0:
                sum = sum + GAP
                matriz[i][j] = sum

    sum = 0
    
    #set GAP first line
    for i in range(num_lin):
        for j in range(num_col):
            if i == 0 and j != 0:
                sum = sum + GAP
                matriz[i][j] = sum

    for i in range(1, num_lin):
        for j in range(1, num_col):

                diag = comp_sequence(matriz, seq_vert, seq_horiz, mismatch, match, i, j)
                left = matriz[i][j] + matriz[i][j-1] + GAP
                up = matriz[i][j] + matriz[i-1][j] + GAP

                values = [diag, left, up]
                max_index, max_value = max(enumerate(values), key=lambda x: x[1])

                if max_index == 0:      #diag
                    matriz_back[i][j] = 0 
                elif max_index == 1:    #left
                    matriz_back[i][j] = 1
                elif max_index == 2:    #up
                    matriz_back[i][j] = 2

                matriz[i][j] = max_value

    return matriz, matriz_back
            

In [376]:
def backtrace(matriz, matriz_back, seq_vert, seq_horiz):
    num_lin = len(matriz)
    num_col = len(matriz[0])
    x = len(seq_horiz) - 1
    y = len(seq_vert) - 1


    matriz_new = def_matriz(seq_vert, seq_horiz)

    soma = 0

    lista_vert = []
    lista_horiz = []


    for i in range(num_lin-1, -1, -1):
        for j in range(num_col-1, -1, -1):

            if i >= 0 and j >= 0 and x >= 0 and y >= 0:
            
                if matriz_back[i][j] == 0:
                    lista_vert.append(seq_vert[y])
                    lista_horiz.append(seq_horiz[x])
                    i = i - 1
                    j = j - 1
                    x = x - 1
                    y = y - 1
                    

                if matriz_back[i][j] == 1:
                    lista_vert.append('-')
                    lista_horiz.append(seq_horiz[x])
                    x = x - 1
                
                if matriz_back[i][j] == 2:
                    lista_vert.append(seq_vert[y])
                    lista_horiz.append('-')
                    y = y - 1
                    i = i - 1
                    j = j + 1


    return lista_vert, lista_horiz

In [377]:
# teste de uso das variáveis
print("Vertical:", read_arq()[0])
print("Horizontal:", read_arq()[1])
print("GAP:", read_arq()[2])
print("Mismatch:", read_arq()[3])
print("Match:", read_arq()[4])

matriz_initial = def_matriz(read_arq()[0], read_arq()[1])
matriz = smith_waterman(matriz_initial, read_arq()[0], read_arq()[1], read_arq()[2], read_arq()[3], read_arq()[4])
matriz_new = backtrace(matriz[0], matriz[1], read_arq()[0], read_arq()[1])

print(matriz[0])
print("\n")
print(matriz[1])
print("\n")
print(matriz_new[0])
print("\n")
print(matriz_new[1])
print("\n")


Vertical: GCUGCCUU
Horizontal: GUCGAC
GAP: -2
Mismatch: -1
Match: 1
[[  0.  -2.  -4.  -6.  -8. -10. -12.]
 [ -2.   1.  -1.  -3.  -5.  -7.  -9.]
 [ -4.  -1.   0.   0.  -2.  -4.  -6.]
 [ -6.  -3.   0.  -1.  -1.  -3.  -5.]
 [ -8.  -5.  -2.  -1.   0.  -2.  -4.]
 [-10.  -7.  -4.  -1.  -2.  -1.  -1.]
 [-12.  -9.  -6.  -3.  -2.  -3.   0.]
 [-14. -11.  -8.  -5.  -4.  -3.  -2.]
 [-16. -13. -10.  -7.  -6.  -5.  -4.]]


[[0. 2. 2. 2. 2. 2. 2.]
 [2. 0. 1. 1. 0. 1. 1.]
 [2. 2. 0. 0. 1. 1. 0.]
 [2. 2. 0. 0. 0. 0. 0.]
 [2. 0. 2. 0. 0. 0. 0.]
 [2. 2. 2. 0. 0. 0. 0.]
 [2. 2. 2. 0. 0. 0. 0.]
 [2. 2. 0. 2. 0. 0. 2.]
 [2. 2. 0. 2. 0. 0. 0.]]


['U', 'U', 'C', 'C', 'G', 'U', 'C', 'G']


['C', 'A', 'G', 'C', '-', 'U', '-', 'G']


